In [1]:
import math
import torch
import gpytorch

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from torch.autograd import Variable

from torch import nn, optim
from gpytorch.kernels import RBFKernel, GridInterpolationKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.random_variables import GaussianRandomVariable

from torch.distributions.normal import Normal
from matplotlib import gridspec

import itertools

from dimension import Real,Integer, Categorical
from bayesian_optimization import BayesianOptimization
%matplotlib inline
%load_ext autoreload
%autoreload 2

torch.cuda.set_device(3)

In [2]:

# We use KISS-GP (kernel interpolation for scalable structured Gaussian Processes)# We us 
# as in https://arxiv.org/pdf/1503.01057.pdf
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        # Near-zero mean
        self.mean_module = ConstantMean(constant_bounds=[-5,5])
        # GridInterpolationKernel over an ExactGP
        self.base_covar_module = RBFKernel(log_lengthscale_bounds=(-5, 6))
        self.covar_module = GridInterpolationKernel(self.base_covar_module, grid_size=200,
                                                    grid_bounds=[(-20, 20), (-20, 20)])
        # Register the log lengthscale as a trainable parametre
        self.register_parameter('log_outputscale', nn.Parameter(torch.Tensor([0])), bounds=(-5,6))
        
    def forward(self,x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        covar_x = covar_x.mul(self.log_outputscale.exp())
        return GaussianRandomVariable(mean_x, covar_x)


In [3]:
#minimize target function
def target(x):
    return -20 * torch.exp(-0.2 * torch.sqrt(0.5* torch.sum(x**2, dim = 1))) - torch.exp(.5 * torch.sum(torch.cos(2 * math.pi * x), dim = 1)) + 20 + torch.exp(torch.ones(x.size(0)).cuda())


# Training data is 6 points in [-5,15] inclusive regularly spaced

x = torch.linspace(-20,20,5)

y =  torch.linspace(-20,20,5)
# 25 samples 
train_x = Variable(torch.stack([x.repeat(y.size(0)), y.repeat(x.size(0),1).t().contiguous().view(-1)],1)).cuda()


# Maximize the negative target function
train_y = Variable(-1*target(train_x)).cuda()

likelihood = GaussianLikelihood(log_noise_bounds=(-8, -6)).cuda()
#model = ExactGPModel(train_x.data, train_y.data, likelihood)
model = GPRegressionModel(train_x.data, train_y.data, likelihood).cuda()
search_space = [Real(-20,20),Real(-20,20)]

In [4]:
#train before optimization
model.train()
likelihood.train()
optimizer = torch.optim.Adam([
        {'params': model.parameters()},  # Includes GaussianLikelihood parameters
    ], lr=0.1)
    # "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 20
for i in range(training_iter):
        # Zero gradients from previous iteration
    optimizer.zero_grad()
        # Output from model
    output = model(train_x)
        # Calc loss and backprop gradients
    print("output",output.mean().size(), "train_y",train_y.size())
    loss = -mll(output, train_y)
    loss.backward()

    print('Iter %d/%d - Loss: %.3f' % (
        i + 1, training_iter, loss.data[0]
        ))
  
    optimizer.step()


output torch.Size([25]) train_y torch.Size([25])
Iter 1/20 - Loss: 161.669
output torch.Size([25]) train_y torch.Size([25])
Iter 2/20 - Loss: 144.824
output torch.Size([25]) train_y torch.Size([25])
Iter 3/20 - Loss: 129.853
output torch.Size([25]) train_y torch.Size([25])
Iter 4/20 - Loss: 116.566
output torch.Size([25]) train_y torch.Size([25])
Iter 5/20 - Loss: 104.786
output torch.Size([25]) train_y torch.Size([25])
Iter 6/20 - Loss: 94.356
output torch.Size([25]) train_y torch.Size([25])
Iter 7/20 - Loss: 85.129
output torch.Size([25]) train_y torch.Size([25])
Iter 8/20 - Loss: 76.971
output torch.Size([25]) train_y torch.Size([25])
Iter 9/20 - Loss: 69.762
output torch.Size([25]) train_y torch.Size([25])
Iter 10/20 - Loss: 63.390
output torch.Size([25]) train_y torch.Size([25])
Iter 11/20 - Loss: 57.757
output torch.Size([25]) train_y torch.Size([25])
Iter 12/20 - Loss: 52.775
output torch.Size([25]) train_y torch.Size([25])
Iter 13/20 - Loss: 48.362
output torch.Size([25]) train

In [5]:
bo = BayesianOptimization(model, likelihood, target, search_space,"discrete_mes")

bo.optimal(10,plot=False)

RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58

In [ ]:
print(bo.x_star)
print(bo.y_star.view(-1))